In [1]:
from sklearn.datasets import load_diabetes,load_breast_cancer
from sklearn.model_selection import train_test_split
import time
import numpy as np
from tqdm import tqdm
import pandas as pd
import scipy as sp

In [16]:
import matplotlib.pyplot as plt

In [24]:
# Imports from cp_implementation
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
import numpy as np
import copy
import time
import snakeviz
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist
import random
from sklearn.datasets import load_diabetes
assert pyro.__version__.startswith('1.8.4')

In [18]:
## Sparse Regression ##
#Well specified?
# train_frac either 1 or 0.7 in their testing
# dataset = "diabetes"
# seed = 100 plus some number that changes with iterations
train_frac = 0.7
dataset = 'diabetes'
seed = 100
def load_traintest_sparsereg(train_frac, dataset,seed):
    #Load dataset
    if dataset =="diabetes":
        x,y = load_diabetes(return_X_y = True)
    elif dataset =="boston":
        x,y = load_boston(return_X_y = True)
    else:
        print('Invalid dataset')
        return

    n = np.shape(x)[0]
    d = np.shape(x)[1]

    #Standardize beforehand (for validity)
    x = (x - np.mean(x,axis = 0))/np.std(x,axis = 0)
    y = (y - np.mean(y))/np.std(y)

    #Train test split
    ind_train, ind_test = train_test_split(np.arange(n), train_size = int(train_frac*n),random_state = seed)
    x_train = x[ind_train]
    y_train = y[ind_train]
    x_test = x[ind_test]
    y_test = y[ind_test]
    
    ### what is y_plot / what is it used for / where do -2 and +2100 come from?
    y_plot = np.linspace(np.min(y_train) - 2, np.max(y_train) + 2,100)
    
    return x_train,y_train,x_test,y_test,y_plot,n,d

In [19]:
x,y,x_test,y_test,y_plot,n,d = load_traintest_sparsereg(train_frac,dataset,seed)

y_plot are all the value of y_hat you will use in your loop.
Goal: use their train test split and run your stuff

In [21]:
print(len(x))
print(len(y))

309
309


In [25]:
# Model and Guide Setup
def model(data):
    # define the hyperparameters that control the Beta prior
    mu0 = torch.zeros(dim, dtype=torch.float64)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.MultivariateNormal(mu0, std0))
    # loop over the observed data
    subset = random.sample(data, int(len(data) / dim))
    for i in range(len(subset)):
        pyro.sample("obs_{}".format(i), dist.Normal(f.dot(data[i][0]), 0.3), obs=data[i][1])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    mu_q = pyro.param("mu_q", copy.deepcopy(prev_mu_q))
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.MultivariateNormal(mu_q, std0))

def train_svi(D_hat, n_steps):
    # setup the optimizer
    adam_params = {"lr": 0.005, "betas": (0.90, 0.999)}
    optimizer = Adam(adam_params)

    # setup the inference algorithm
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    # do gradient steps
    for step in range(n_steps):
        loss = svi.step(D_hat)
        if loss < 1e-5:
            break
    
    mu_q = pyro.param("mu_q")
    return mu_q

In [ ]:
global prev_mu_q
prev_mu_q = torch.zeros(dim, dtype=torch.float64)
std0 = torch.eye(dim, dtype=torch.float64) * 0.3

def run_svi(X_train, Y_train, y_plot, threshold=0.9):
    # Run SVI on the Data
    y_hat = max(Y_train)
    y_bottom = min(Y_train)
    print(y_hat)
    print(y_bottom)
    conformal_set = []
    start = time.time()
    for y_hat in y_plot: # changed from while loop
        pyro.clear_param_store()
        # Create D_hat
        D_hat = list(zip(X_train[:-1], Y_train))
        D_hat.append((X_train[-1], y_hat))
        
        # Train SVI
        mu_q = train_svi(D_hat, 4)
        prev_mu_q = mu_q
        
        # Calculate rank of y_hat
        rank = [(abs(sum(D_hat[i][0] * mu_q) - D_hat[i][1]).detach().numpy()) for i in range(len(D_hat))]
        y_hat_rank = rank[-1]
        
        # Add to conformal set if in not in bottom 10 percent of probabilities
        if np.count_nonzero(y_hat_rank > rank) / len(rank) < threshold:
            conformal_set.append(copy.deepcopy(y_hat))
            print(f"{y_hat} Added")
        else:
            print(f"{y_hat} Not added")
            
        y_hat -= decrease_size
    conformal_set = [min(conformal_set), max(conformal_set)]
    end = time.time()

    print(f"Conformal Set: [{float(conformal_set[0])}, {float(conformal_set[1])}]")
    print(f"Length: {float(conformal_set[1] - conformal_set[0])}")
    print(f"Y[-1]: {Y[-1]}")
    if Y[-1] >= conformal_set[0] and Y[-1] <= conformal_set[1]:
        print("Y[-1] is covered")
    else:
        print("Y[-1] is Not covered")
    print(f"Elapsed Time: {end - start}")

In [31]:
def normal_loglikelihood(y,x):
            return sp.stats._distn_infrastructure.pdf(y,loc = np.dot(beta_post[j],x.transpose())+ intercept_post[j],scale = sigma_post[j]) #compute likelihood samples

In [32]:
normal_loglikelihood(y, x)

AttributeError: module 'scipy.stats._distn_infrastructure' has no attribute 'pdf'